We see that there are many missing values and they can be taken from the internet. I can proceed in that direction.

In [16]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import mysql.connector

In [2]:
df_netflix_movie_data = pd.read_csv('./Datasets/netflix_titles.csv', lineterminator = '\n')
df_prime_movie_data = pd.read_csv('./Datasets/amazon_prime_titles.csv', lineterminator = '\n')
df_disney_plus_movie_data = pd.read_csv('./Datasets/disney_plus_titles.csv', lineterminator = '\n')
df_hulu_movie_data = pd.read_csv('./Datasets/hulu_titles.csv', lineterminator = '\n')

In [3]:
missing_value_netflix = df_netflix_movie_data.isna().sum()
print(missing_value_netflix)

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64


In [4]:
missing_value_prime = df_prime_movie_data.isna().sum()
print(missing_value_prime)

show_id            0
type               0
title              0
director        2083
cast            1233
country         8996
date_added      9513
release_year       0
rating           337
duration           0
listed_in          0
description        0
dtype: int64


In [5]:
missing_value_disney_plus = df_disney_plus_movie_data.isna().sum()
print(missing_value_disney_plus)

show_id           0
type              0
title             0
director        473
cast            190
country         219
date_added        3
release_year      0
rating            3
duration          0
listed_in         0
description       0
dtype: int64


In [6]:
missing_value_hulu = df_hulu_movie_data.isna().sum()
print(missing_value_hulu)

show_id            0
type               0
title              0
director        3070
cast            3073
country         1453
date_added        28
release_year       0
rating           520
duration         479
listed_in          0
description        4
dtype: int64


As we can see clearly, above only columns director, cast, country, date_added, rating, duration. Rather than deleting them. If we can get those values back in the dataset it would be best.
I can use information present on the website imdb to get the data and fill it. I can use web scrapping to get the data and fill it in the dataset.

But before that I can check data. One dataset row may have data of the for the other dataset missing row values. Let's first check for the 

In [ ]:
conn = mysql.connector.connect(
    host="127.0.0.1",  # Use IP instead of 'localhost'
    user="Himanshu",
    password="Hustling@2000",
)

Connected Successfully!
